In [1]:
from slap import * 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.offline as po
import plotly.graph_objs as go
from plotly import tools
import load
import descriptions

po.init_notebook_mode(connected=True)

# Contents


1. The datasets
2. av_tumour
3. av_patient <br/>
&nbsp; 3.1  merge av_tumour and av_patient
4. sact_regimen
5. sact_tumour <br/>
&nbsp; 5.1  merge sact_tumour sact_regimen
6. sact_outcome <br/>
&nbsp; 6.1 merge sact_outcome and sact_regimen
7. sact_cycle
8. sact_patient
9. sact_drug_detail
10. Patient pathway/single view of patient <br/>
&nbsp; 10.1 merging all datasets <br/>
&nbsp; 10.2 patient pathways <br/>

# 1. The datasets

This is a table of all the columns in each dataset, there are a lot so it might be useful to see all of them in one table

**can scroll this table left/right**

In [5]:
datadict = pd.ExcelFile('Simulacrum-data-dictionary.xlsx')
sheets = datadict.sheet_names[1:]
DF = pd.DataFrame()

for sheet in sheets:
    df = pd.read_excel(datadict,sheet)
    df[sheet] = df['Field name']
    DF = pd.concat([DF,df[sheet]], axis=1)
    DF = DF.replace(np.nan, '', regex=True)
DF

,AV_PATIENT,AV_TUMOUR,SACT PATIENT,SACT TUMOUR,SACT REGIMEN,SACT OUTCOME,SACT CYCLE,SACT DRUG_DETAIL
0,PATIENTID,TUMOURID,MERGED_PATIENT_ID,MERGED_TUMOUR_ID,MERGED_REGIMEN_ID,MERGED_OUTCOME_ID,MERGED_CYCLE_ID,MERGED_DRUG_DETAIL_ID
1,SEX,PATIENTID,LINK_NUMBER,MERGED_PATIENT_ID,MERGED_TUMOUR_ID,MERGED_REGIMEN_ID,MERGED_REGIMEN_ID,MERGED_CYCLE_ID
2,LINKNUMBER,DIAGNOSISDATEBEST,,CONSULTANT_SPECIALITY_CODE,HEIGHT_AT_START_OF_REGIMEN,DATE_OF_FINAL_TREATMENT,CYCLE_NUMBER,ORG_CODE_OF_DRUG_PROVIDER
3,ETHNICITY,SITE_ICD10_O2,,PRIMARY_DIAGNOSIS,WEIGHT_AT_START_OF_REGIMEN,REGIMEN_MOD_DOSE_REDUCTION,START_DATE_OF_CYCLE,ACTUAL_DOSE_PER_ADMINISTRATION
4,DEATHCAUSECODE_1A,SITE_ICD10_O2_3CHAR,,MORPHOLOGY_CLEAN,INTENT_OF_TREATMENT,REGIMEN_MOD_TIME_DELAY,OPCS_PROCUREMENT_CODE,OPCS_DELIVERY_CODE
5,DEATHCAUSECODE_1B,MORPH_ICD10_O2,,,DATE_DECISION_TO_TREAT,REGIMEN_MOD_STOPPED_EARLY,PERF_STATUS_START_OF_CYCLE,ADMINISTRATION_ROUTE
6,DEATHCAUSECODE_1C,BEHAVIOUR_ICD10_O2,,,START_DATE_OF_REGIMEN,REGIMEN_OUTCOME_SUMMARY,MERGED_PATIENT_ID,ADMINISTRATION_DATE
7,DEATHCAUSECODE_2,T_BEST,,,MAPPED_REGIMEN,MERGED_PATIENT_ID,MERGED_TUMOUR_ID,DRUG_GROUP
8,DEATHCAUSECODE_UNDERLYING,N_BEST,,,CLINICAL_TRIAL,MERGED_TUMOUR_ID,,MERGED_PATIENT_ID
9,DEATHLOCATIONCODE,M_BEST,,,CHEMO_RADIATION,,,MERGED_TUMOUR_ID


Diagram of how the datasets are linked:

<img src="docs/tablestructure.jpeg" alt="ts" width="500"/>

The data can be downloaded from https://simulacrum.healthdatainsight.org.uk/requesting-data/

# 2. Tumour

In [6]:
av_tumour = load.load_table('av_tumour', add_descriptions=True) #add_descriptions=True adds columns of
                                                                #descriptions to the codes
display(av_tumour.head())
print(av_tumour.shape)

,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,SITE_ICD10_O2,SITE_ICD10_O2_3CHAR,MORPH_ICD10_O2,BEHAVIOUR_ICD10_O2,T_BEST,N_BEST,M_BEST,...,CREG_CODE_DESC,ER_STATUS_DESC,PR_STATUS_DESC,HER2_STATUS_DESC,CANCERCAREPLANINTENT_DESC,PERFORMANCESTATUS_DESC,CNS_DESC,ACE27_DESC,LATERALITY_DESC,HISTOLOGY_DESC
0,10000001,10000001,2015-11-06,C443,C44,8090,3,NaN,NaN,NaN,...,Oxford Cancer Intelligence Unit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LEFT,BASAL CELL CARCINOMA NOS
1,10000002,10000002,2015-10-06,C449,C44,8090,3,NaN,NaN,NaN,...,North West Cancer Intelligence Service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT APPLICABLE,BASAL CELL CARCINOMA NOS
2,10000003,10000003,2013-01-30,C449,C44,8090,3,NaN,NaN,NaN,...,Northern & Yorkshire Cancer Registry & Informa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT APPLICABLE,BASAL CELL CARCINOMA NOS
3,10000004,10000004,2015-09-15,C449,C44,8090,3,NaN,NaN,NaN,...,Thames Cancer Registry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT APPLICABLE,BASAL CELL CARCINOMA NOS
4,10000005,10000005,2014-11-01,C449,C44,8090,3,NaN,NaN,NaN,...,Northern & Yorkshire Cancer Registry & Informa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT APPLICABLE,BASAL CELL CARCINOMA NOS


(1402817, 51)


Unique diagnosis years:

In [7]:
av_tumour["DIAGNOSISDATEBEST"].apply(lambda x: x.year).unique()

array([2015, 2013, 2014])

top 20 diagnosed tumours:

In [8]:
av_tumour['SITE_ICD10_O2_3CHAR_DESC'].value_counts()[:20]

Other malignant neoplasms of skin                       350130
Malignant neoplasm of breast                            133907
Malignant neoplasm of prostate                          119347
Malignant neoplasm of bronchus and lung                 102350
Carcinoma in situ of cervix uteri                        76678
Malignant neoplasm of colon                              65825
Malignant melanoma of skin                               38155
Malignant neoplasm of rectum                             27163
Malignant neoplasm of kidney, except renal pelvis        26195
Carcinoma in situ of other and unspecified sites         25193
Malignant neoplasm of bladder                            24643
Malignant neoplasm of corpus uteri                       21849
Malignant neoplasm of pancreas                           21344
Malignant neoplasm of oesophagus                         20996
Carcinoma in situ of breast                              19777
Diffuse non-Hodgkin's lymphoma                         

Can compare this with:

https://www.cancerresearchuk.org/health-professional/cancer-statistics/incidence/common-cancers-compared#heading-Zero

We have the top diagnosed cancer as skin cancer, this may be due to single patients having multiple melanomas of skin. 

Making some stacked bar plots of the tumour features:

In [13]:
stacked_barplot(av_tumour, 'SITE_ICD10_O2_3CHAR', 'AGE')

The colours on this graph correspond to tumour site

We can compare this graph with a graph from cancerresearchuk:

https://www.cancerresearchuk.org/health-professional/cancer-statistics/incidence/age#heading-Zero

The distribution is similar except the peak at around 25 years due to D06, you can convert D06 into actual words:

In [8]:
descriptions.get_descriptions('D06', 'icd')[0]

'CARCINOMA-IN-SITU OF CERVIX UTERI'

The peak due to cervical cancer is also apparent in this graph from cancerresearchuk:

https://www.cancerresearchuk.org/health-professional/cancer-statistics/statistics-by-cancer-type/cervical-cancer/incidence#heading-One

<br/>
<br/>

Can also plot other columns of av_tumour, for example behaviour:

In [16]:
for i in range(10):
    print(i,descriptions.get_descriptions(str(i),'behaviour')[0])
stacked_barplot(av_tumour, 'SITE_ICD10_O2_3CHAR', 'BEHAVIOUR_ICD10_O2',tickmode='linear')

0 BENIGN
1 UNCERTAIN
2 IN SITU
3 MALIGNANT
4 nan
5 MICRO-INVASIVE
6 MALIGNANT, METASTATIC/ SECONDARY SITE
7 nan
8 nan
9 MALIGNANT, UNCERTAIN WHETHER PRIMARY OR METASTATIC


The meaning of behaviour:

### Errors in av_tumour

There are 54161 entries in the av_tumour dataset with errenous 3 character IDC10 codes. Examples of such errors are shown in the data frame below. 

In [21]:
incorrect = av_tumour['SITE_ICD10_O2'].apply(lambda s: str(s)[0:3]) \
            != av_tumour['SITE_ICD10_O2_3CHAR']
print(incorrect.value_counts()[1],"errenous entries. ")
display(av_tumour[incorrect].tail(3))


54161 errenous entries. 


,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,SITE_ICD10_O2,SITE_ICD10_O2_3CHAR,MORPH_ICD10_O2,BEHAVIOUR_ICD10_O2,T_BEST,N_BEST,M_BEST,...,CREG_CODE_DESC,ER_STATUS_DESC,PR_STATUS_DESC,HER2_STATUS_DESC,CANCERCAREPLANINTENT_DESC,PERFORMANCESTATUS_DESC,CNS_DESC,ACE27_DESC,LATERALITY_DESC,HISTOLOGY_DESC
1402814,220052225,220045122,2015-01-30,D075,D01,8140,2,NaN,NaN,NaN,...,Oxford Cancer Intelligence Unit,NaN,NaN,NaN,Curative,Able to carry out all normal activity without ...,NaN,NaN,NOT APPLICABLE,ADENOCARCINOMA IN SITU
1402815,220052228,220010521,2015-11-02,C051,C61,8000,3,NaN,2,1,...,West Midlands Cancer Intelligence Unit,NaN,NaN,NaN,Non Curative,"Able to walk and capable of all self care, but...",NaN,NaN,NOT APPLICABLE,NEOPLASM MALIGNANT
1402816,220052229,220014422,2015-10-27,C441,C64,8046,3,2a,2,1,...,Eastern Cancer Registration & Information Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LEFT,NON SMALL CELL CARCINOMA OF LUNG


# 3. Patient

In [2]:
av_patient = load.load_table('av_patient', add_descriptions=True)
display(av_patient.head())

,PATIENTID,SEX,LINKNUMBER,ETHNICITY,DEATHCAUSECODE_1A,DEATHCAUSECODE_1B,DEATHCAUSECODE_1C,DEATHCAUSECODE_2,DEATHCAUSECODE_UNDERLYING,DEATHLOCATIONCODE,...,VITALSTATUSDATE,SEX_DESC,ETHNICITY_DESC,DEATHLOCATIONCODE_DESC,DEATHCAUSECODE_1A_DESC,DEATHCAUSECODE_1B_DESC,DEATHCAUSECODE_1C_DESC,DEATHCAUSECODE_2_DESC,DEATHCAUSECODE_UNDERLYING_DESC,NEWVITALSTATUS_DESC
0,10000001,2,810000001,A,NaN,NaN,NaN,NaN,NaN,NaN,...,2017-01-17,FEMALE,WHITE BRITISH,NaN,,,,,,Alive
1,10000002,2,810000002,Z,NaN,NaN,NaN,NaN,NaN,NaN,...,2017-01-14,FEMALE,NOT STATED,NaN,,,,,,Alive
2,10000003,1,810000003,A,NaN,NaN,NaN,NaN,NaN,NaN,...,2017-01-17,MALE,WHITE BRITISH,NaN,,,,,,Alive
3,10000004,1,810000004,A,NaN,NaN,NaN,NaN,NaN,NaN,...,2017-01-13,MALE,WHITE BRITISH,NaN,,,,,,Alive
4,10000005,2,810000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017-01-16,FEMALE,NaN,NaN,,,,,,Alive


In [3]:
av_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322100 entries, 0 to 1322099
Data columns (total 21 columns):
PATIENTID                         1322100 non-null int64
SEX                               1322100 non-null category
LINKNUMBER                        1322100 non-null int64
ETHNICITY                         1192249 non-null category
DEATHCAUSECODE_1A                 330280 non-null object
DEATHCAUSECODE_1B                 98085 non-null object
DEATHCAUSECODE_1C                 18106 non-null object
DEATHCAUSECODE_2                  141982 non-null object
DEATHCAUSECODE_UNDERLYING         327910 non-null object
DEATHLOCATIONCODE                 330381 non-null category
NEWVITALSTATUS                    1322100 non-null category
VITALSTATUSDATE                   1322100 non-null datetime64[ns]
SEX_DESC                          1322100 non-null object
ETHNICITY_DESC                    1192249 non-null object
DEATHLOCATIONCODE_DESC            323235 non-null object
DEATHCAUSECO

Which years are the data from?

In [4]:
av_patient['VITALSTATUSDATE'].apply(lambda x: x.year).unique()

array([2017, 2016, 2015, 2013, 2014, 1900])

Which years are the data from?

In [5]:
av_patient['VITALSTATUSDATE'].apply(lambda x: x.year).unique()

array([2017, 2016, 2015, 2013, 2014, 1900])

Frequency of the top 20 death causes:

## Frequency plots of Patient data 

In [6]:
#group by sex and count the type of deaths

useable_groupings = ['DEATHCAUSECODE_UNDERLYING_DESC', 'ETHNICITY_DESC',
                     'DEATHLOCATIONCODE_DESC',]
av_patient_frequency(av_patient,useable_groupings[2])


You can compare this graph with:

https://www.cancerresearchuk.org/health-professional/cancer-statistics/mortality/common-cancers-compared#heading-Zero

Which is from 2015

### Errors from av_patient
There are a number of entries that can be described as 'schrodinger's patients' who are both dead and alive at the same time. These quantum patients have probably errenously been generated by the simulacrum algorithm. 

non null `"DEATHCAUSECODE_UNDERLYING"` and `"NEWVITALSTATUS" != "D"`:

In [8]:
dead_alive = av_patient[(av_patient["DEATHCAUSECODE_UNDERLYING"].notnull()) \
                        & (av_patient["NEWVITALSTATUS"] != "D")]
print("There are",len(dead_alive),"such quantum patients")

There are 179511 such quantum patients


### 3.2 merge av_tumour and av_patient

In [18]:
tumour_patient = pd.merge(av_tumour,av_patient,on='PATIENTID')
print(av_tumour.shape,av_patient.shape,tumour_patient.shape) 

(1402817, 51) (1322100, 21) (1402817, 71)


create column with days between date of vital status taken and date of diagnosis

In [19]:
tumour_patient['VITAL - DIAGNOSIS (days)'] = tumour_patient['VITALSTATUSDATE'] - tumour_patient['DIAGNOSISDATEBEST']
tumour_patient['VITAL - DIAGNOSIS (days)'] = [i.days for i in tumour_patient['VITAL - DIAGNOSIS (days)']]
tumour_patient = tumour_patient[tumour_patient['VITAL - DIAGNOSIS (days)'] >= 0] #some have vital status in 1900

print("stats for days being alive")
tumour_patient['VITAL - DIAGNOSIS (days)'].describe()


stats for days being alive


count    1.402784e+06
mean     7.674820e+02
std      4.079234e+02
min      0.000000e+00
25%      4.660000e+02
50%      7.690000e+02
75%      1.105000e+03
max      1.484000e+03
Name: VITAL - DIAGNOSIS (days), dtype: float64

# 4. Sact_regimen

https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python - loading sim_sact_regimen

there are multiple regimens for some tumours

In [14]:
sact_regimen = pd.read_csv('../simulacrum_release_v1.1.0/sim_sact_regimen.csv',encoding="ISO-8859-1")
print(sact_regimen.shape)
display(sact_regimen.head())

(471919, 12)


,MERGED_REGIMEN_ID,MERGED_TUMOUR_ID,HEIGHT_AT_START_OF_REGIMEN,WEIGHT_AT_START_OF_REGIMEN,INTENT_OF_TREATMENT,DATE_DECISION_TO_TREAT,START_DATE_OF_REGIMEN,MAPPED_REGIMEN,CLINICAL_TRIAL,CHEMO_RADIATION,MERGED_PATIENT_ID,BENCHMARK_GROUP
0,10000001,10000001,-1.0,NaN,N,2015-03-08,2015-03-08,DEGARELIX,NaN,N,10000235,HORMONES
1,10000002,10000002,NaN,NaN,P,2016-07-09,2016-07-10,BICALUTAMIDE + GOSERELIN,NaN,N,10000315,HORMONES
2,10000003,10000003,NaN,NaN,P,2014-01-09,2014-01-09,EXEMESTANE,02,N,10000337,HORMONES
3,10000004,10000004,NaN,NaN,NaN,2015-03-17,2015-03-23,Rituximab,N,NaN,10000480,RITUXIMAB
4,10000005,10000005,0.0,NaN,C,2012-12-30,2013-03-11,Hydroxycarbamide,N,N,10000533,HYDROXYCARBAMIDE


# 5. Sact_tumour

tumours that have received systemic anti-cancer therapy

In [ ]:
sact_tumour = load.load_table('sact_tumour')
sact_tumour.head()

### 5.1 merge sact_tumour sact_regimen

In [ ]:
tumour_regimen = pd.merge(sact_tumour,sact_regimen,on='MERGED_TUMOUR_ID')
print(sact_tumour.shape, sact_regimen.shape, tumour_regimen.shape) 

In [ ]:
tumour_regimen.head()

In [ ]:
#only include top20 benchmark group and top30 primary diagnosis in tumour_regimen
#this is beccause there are 809 different benchmarks and 700 different diagnosis

top20bench = tumour_regimen['BENCHMARK_GROUP'].value_counts()[:20].keys()
top30diag = tumour_regimen['PRIMARY_DIAGNOSIS'].value_counts()[:30].keys()

topbenchdiag = tumour_regimen.loc[(tumour_regimen['BENCHMARK_GROUP'].isin(top20bench)) & 
                                  (tumour_regimen['PRIMARY_DIAGNOSIS'].isin(top30diag))]

In [ ]:
stacked_barplot(topbenchdiag,'BENCHMARK_GROUP', 'PRIMARY_DIAGNOSIS')

This is a frequency plot of primary diagnosis of patients with the different colours as the BENCHMARK_GROUP(type of regimen i think) given to that patient.

this only shows the top 30 PRIMARY_DIAGNOSIS and top 30 BENCHMARK_GROUP

# 6. sact_outcome

In [ ]:
sact_outcome = pd.read_csv('simulacrum_release_v1.1.0/sim_sact_outcome.csv')
sact_outcome.head()

In [ ]:
sact_outcome.shape

### 6.1 merge sact_outcome and sact_regimen

In [ ]:
outcome_regimen = pd.merge(sact_regimen,sact_outcome,on='MERGED_REGIMEN_ID')
print(sact_outcome.shape, sact_regimen.shape, outcome_regimen.shape) 

In [ ]:
outcome_regimen.head()

In [ ]:
#only include top 20 bench
top20bench = outcome_regimen['BENCHMARK_GROUP'].value_counts()[:20].keys()

top_outcome_regimen = outcome_regimen.loc[outcome_regimen['BENCHMARK_GROUP'].isin(top20bench)]

stacked_barplot(top_outcome_regimen,'BENCHMARK_GROUP', 'REGIMEN_OUTCOME_SUMMARY', tickmode='linear')

This is a frequency plot of REGIMEN_OUTCOME_SUMMARY with the different colours as the BENCHMARK_GROUP (drugs prescribed I think)

What the numbers mean:

In [ ]:
for i in range(10):
    print(i,descriptions.get_descriptions(str(i),'regimenoutcome')[0])


# 7. sact_cycle

In [ ]:
sact_cycle = pd.read_csv('simulacrum_release_v1.1.0/sim_sact_cycle.csv', low_memory=False)
display(sact_cycle.head())

print('shape ',sact_cycle.shape)

# 8. sact_patient

In [ ]:
sact_patient = pd.read_csv('simulacrum_release_v1.1.0/sim_sact_patient.csv')
display(sact_patient.head())

print('shape ',sact_patient.shape)


# 9. sact_drug_detail

In [11]:
drug_detail = pd.read_csv('../simulacrum_release_v1.1.0/sim_sact_drug_detail.csv')
display(drug_detail.head())

,MERGED_DRUG_DETAIL_ID,MERGED_CYCLE_ID,ORG_CODE_OF_DRUG_PROVIDER,ACTUAL_DOSE_PER_ADMINISTRATION,OPCS_DELIVERY_CODE,ADMINISTRATION_ROUTE,ADMINISTRATION_DATE,DRUG_GROUP,MERGED_PATIENT_ID,MERGED_TUMOUR_ID,MERGED_REGIMEN_ID
0,10000001,10000001,EKD02,10.0,NaN,1.0,2015-06-07,STEROID,10000235,10000001,10000001
1,10000002,10000002,EGU02,30.0,NaN,2.0,NaN,NOT CHEMO,10000315,10000002,10000002
2,10000003,10000003,EGU05,1000.0,NaN,1.0,NaN,RITUXIMAB,10000337,10000003,10000003
3,10000004,10000004,EXO,8.0,X723,2.0,2015-03-23,NOT CHEMO,10000480,10000004,10000004
4,10000005,10000005,ETD02,1000.0,NaN,2.0,2013-03-11,NOT CHEMO,10000533,10000005,10000005


In [12]:
drugs = drug_detail['DRUG_GROUP'].unique()
sorted(drugs)

['ABIRATERONE',
 'ACELARIN',
 'ADI-PEG 20',
 'AFATINIB',
 'AFLIBERCEPT',
 'ALDESLEUKIN',
 'ALEMTUZUMAB',
 'AMSACRINE',
 'ANAGRELIDE',
 'ANASTROZOLE',
 'ARSENIC TRIOXIDE',
 'ASPARAGINASE',
 'ATEZOLIZUMAB',
 'ATG',
 'AVELUMAB',
 'AXITINIB',
 'AZACITIDINE',
 'BCG',
 'BENDAMUSTINE',
 'BEVACIZUMAB',
 'BEXAROTENE',
 'BICALULTAMIDE',
 'BICALUTAMIDE',
 'BLEOMYCIN',
 'BLINATUMOMAB',
 'BORTEZOMIB',
 'BOSUTINIB',
 'BRENTUXIMAB',
 'BRENTUXIMAB VEDOTIN',
 'BUSULFAN',
 'CABAZITAXEL',
 'CABOZANTINIB',
 'CALCIUM FOLINATE',
 'CAPECITABINE',
 'CARBOPLATIN',
 'CARFILZOMIB',
 'CARMUSTINE',
 'CEDIRANIB',
 'CERITINIB',
 'CETIRIZINE',
 'CETRORELIX ACETATE',
 'CETUXIMAB',
 'CHLORAMBUCIL',
 'CISPLATIN',
 'CLADRIBINE',
 'CLOFARABINE',
 'COPANLISIB',
 'CRIZOTINIB',
 'CYCLOPHOSPHAMIDE',
 'CYPROTERONE',
 'CYTARABINE',
 'DABRAFENIB',
 'DACARBAZINE',
 'DACTINOMYCIN',
 'DARATUMUMAB',
 'DARBOPOETIN',
 'DASATINIB',
 'DAUNORUBICIN',
 'DECITABINE',
 'DEGARELIX',
 'DENOSUMAB',
 'DEXAMETHASONE',
 'DEXRAZOXANE (SAVENE)',
 '

there are some repeated drugs due to spelling mistakes e.g.  'GEMTUZUMAB OZOGAMICIN' 'GEMTUZUMAB OZOGAMYCIN'

A basic fix to these spelling mistakes is to use a sequence similarity measure:

In [13]:
from difflib import SequenceMatcher

def similar(a, b):
    """returns a similarity measure in [0,1]
    between two strings a, b"""
    return SequenceMatcher(None, a, b).ratio()

#examples
for spelling1, spelling2 in zip(['IBRITINIB', 'GEMTUZUMAB OZOGAMICIN', 'FLUCONAZOLE', 'MASITINIB'],
                                ['IBRUTANIB', 'GEMTUZUMAB OZOGAMYCIN', 'FLUOROURACIL', 'AXITINIB']):
    print(similar(spelling1,spelling2), spelling1 + ", " + spelling2)



0.7777777777777778 IBRITINIB, IBRUTANIB
0.9523809523809523 GEMTUZUMAB OZOGAMICIN, GEMTUZUMAB OZOGAMYCIN
0.43478260869565216 FLUCONAZOLE, FLUOROURACIL
0.8235294117647058 MASITINIB, AXITINIB


The problem here is `MASITINIB,AXITINIB` are similar but not the same drug.

To fix this we can split the word into three and calculate the similarities between the first/second parts:

In [14]:
def similar2(a, b):
    """returns similarity measures in [0,1]
    between the first and second half 
    of strings a, b"""
    
    len_a = int(len(a)/2)
    len_b = int(len(b)/2)
    first_half = SequenceMatcher(None, a[:len_a], b[:len_b]).ratio()
    second_half = SequenceMatcher(None, a[len_a:], b[len_b:]).ratio()
    
    return first_half, second_half

#same examplesthird
for spelling1, spelling2 in zip(['IBRITINIB', 'GEMTUZUMAB OZOGAMICIN', 'FLUCONAZOLE', 'MASITINIB'],
                                ['IBRUTANIB', 'GEMTUZUMAB OZOGAMYCIN', 'FLUOROURACIL', 'AXITINIB']):
    print(similar2(spelling1,spelling2), spelling1 + "," + spelling2)

(0.75, 0.8) IBRITINIB,IBRUTANIB
(1.0, 0.9090909090909091) GEMTUZUMAB OZOGAMICIN,GEMTUZUMAB OZOGAMYCIN
(0.7272727272727273, 0.3333333333333333) FLUCONAZOLE,FLUOROURACIL
(0.5, 0.8888888888888888) MASITINIB,AXITINIB


now choose a threshold such that both the first and second half of each spelling are similar. This will decide if it's they are spelling mistakes of the same drug.

The identified spelling mistakes are:

In [15]:
threshold = 0.7
spelling_mistakes = []

for spelling1 in drugs:
    
    similar_spelling1 = [spelling1]
    
    for spelling2 in drugs:
        
        if (all(np.array(similar2(spelling1,spelling2))>0.7) and  spelling1 != spelling2):
            
            similar_spelling1.append(spelling2)
    
    if len(similar_spelling1) != 1:
        print(similar_spelling1)
    spelling_mistakes.append(similar_spelling1)
    
spelling_mistakes = list(set([tuple(sorted(s_m)) for s_m in spelling_mistakes]))

['OCTRETOTIDE', 'OCTREOTIDE']
['MITOMYCIN', 'MYTOMYCIN']
['IDELALISIB', 'IDEALISIB']
['IBRUTINIB', 'IBRUTANIB', 'IBRITINIB']
['GEFITINIB', 'GEFETINIB']
['BICALUTAMIDE', 'BICALULTAMIDE']
['HYDROXYCARBAMIDE', 'HYDROXYCARBIMIDE']
['LIPOSOMAL DOXORUBICIN', 'LIPOSOMAL DAUNORUBICIN']
['ELOTUZUMAB', 'VELTUZUMAB']
['NINTEDANIB', 'NINTENDINIB']
['LIPOSOMAL DOXORUBICIN (CAELYX)', 'LIPOSOMAL DOXORUBICIN (MYOCET)']
['GEMTUZUMAB OZOGAMYCIN', 'GEMTUZUMAB OZOGAMICIN']
['BICALULTAMIDE', 'BICALUTAMIDE']
['NINTENDINIB', 'NINTEDANIB']
['GEFETINIB', 'GEFITINIB']
['MYTOMYCIN', 'MITOMYCIN']
['IBRUTANIB', 'IBRUTINIB', 'IBRITINIB']
['VELTUZUMAB', 'ELOTUZUMAB']
['IDEALISIB', 'IDELALISIB']
['GEMTUZUMAB OZOGAMICIN', 'GEMTUZUMAB OZOGAMYCIN']
['IBRITINIB', 'IBRUTINIB', 'IBRUTANIB']
['LIPOSOMAL DAUNORUBICIN', 'LIPOSOMAL DOXORUBICIN']
['HYDROXYCARBIMIDE', 'HYDROXYCARBAMIDE']
['OCTREOTIDE', 'OCTRETOTIDE']


There are two misidentified spelling mistake, this algorithm can be improved more in detail but it will not be important as there are only about 0.3\% spelling mistakes

now to replace the drugs with a single spelling:

In [16]:
#create a map from multiple spelling to single spelling:

#all 
single_spelling = {}
for s_m in spelling_mistakes:
    for s in s_m:
        single_spelling[s]=s_m[0]
#example
single_spelling['GEFITINIB']

'GEFETINIB'

apply this map to the drug_group column

In [18]:
drug_detail['DRUG_GROUP_CORRECT'] = drug_detail['DRUG_GROUP'].apply(lambda x: single_spelling[x])

In [19]:
(drug_detail['DRUG_GROUP_CORRECT'] == drug_detail['DRUG_GROUP']).value_counts()

True     3531523
False      13061
dtype: int64

there are about 0.3\% spelling mistakes

# 10. Full patient pathway

To get the full patient pathway we have to merge all datasets

All merges are an inner join.

Only patients that has undertaken a regimen are kept in the end because patients without a regimen are not joined in an inner join.

In [ ]:
z = sact_patient.rename(index=str,columns={'LINK_NUMBER':'LINKNUMBER'})

a = pd.merge(av_tumour, av_patient, on=['PATIENTID', 'SEX', 'LINKNUMBER'])
b = pd.merge(a, z, on='LINKNUMBER')
c = pd.merge(b, sact_tumour, on="MERGED_PATIENT_ID")
# clean the data by removing entries where 1st 3 chars of SITE_ICD10 a nd PRIMARY_DIAGNOSIS are not equal
c_cleaned = c[ c['SITE_ICD10_O2'].apply(lambda s: str(s)[0:3]) == c['PRIMARY_DIAGNOSIS'].apply(lambda s: str(s)[0:3]) ]
d = pd.merge(c_cleaned, sact_regimen, on='MERGED_TUMOUR_ID')
e = pd.merge(d, sact_outcome, on='MERGED_REGIMEN_ID')
f = pd.merge(e, sact_cycle, on='MERGED_REGIMEN_ID')
g = pd.merge(f, drug_detail, on='MERGED_CYCLE_ID')

#remove patients with any number of nans in the columns 'ADMINISTRATION_DATE' or 'CYCLE_NUMBER'
patients_noadmindate = g[g['ADMINISTRATION_DATE'].isnull()]["PATIENTID"]
patients_nocycledate = g[g['CYCLE_NUMBER'].isnull()]["PATIENTID"]
g = g[~g["PATIENTID"].isin(patients_noadmindate)]
g = g[~g["PATIENTID"].isin(patients_nocycledate)]

now sort the dates

In [ ]:
%%time
full_patient_pathways = g \
                    .sort_values(by = 
                    ['PATIENTID', 'ADMINISTRATION_DATE', 'START_DATE_OF_CYCLE', 
                     'START_DATE_OF_REGIMEN', 'DIAGNOSISDATEBEST']) 
full_patient_pathways.to_pickle('full_patient_pathways.pkl')

In [ ]:
full_patient_pathways.head(10)

Which collumns don't have a lot of NaNs?

In [ ]:
import missingno as msno

#plots the number of not NaN in each column
msno.bar(full_patient_pathways)

In [21]:
xaxis=dict(
        title='x Axis',
        

{'title': 'x Axis',
 'titlefont': {'family': 'Courier New, monospace',
  'size': 18,
  'color': '#7f7f7f'}}